In [ ]:
import random
import json
from tqdm import tqdm
import unicodedata

import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader

# local modules
from ner_tokenizer_bio import NER_tokenizer_BIO
from bert_for_token_classification_pl import BertForTokenClassification_pl

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [ ]:
# インテントの種類数 (None=0, LED_ON=1, LED_OFF=2, READ_THERMO=3, OPEN=4, CLOSE=5, SET_TEMP=6)
NUM_INTENT_LABELS = 7

# スロットの種類数 (COL=1, COLLTDEV=2, LOC=3, ONOFFDEV=4, OPENABLE=5, TEMPDEV=6, TEMPERTURE_NUM=7, THMDEV=8)
NUM_ENTITY_TYPE = 8

# データのロード
dataset = json.load(open('data/nlp_data.json','r'))

# カテゴリーをラベルに変更、文字列の正規化する。
for sample in dataset:
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])

# データセットの分割
random.shuffle(dataset)
dataset = dataset[:10000]
n       = len(dataset)
n_train = int(n*0.6)
n_val   = int(n*0.2)
dataset_train = dataset[:n_train]
dataset_val   = dataset[n_train:n_train+n_val]
dataset_test  = dataset[n_train+n_val:]

In [ ]:
def create_dataset(tokenizer, dataset, max_length):
    """
    データセットをデータローダに入力できる形に整形。
    """
    dataset_for_loader = []
    for sample in dataset:
        text = sample['text']
        entities = sample['entities']
        encoding = tokenizer.encode_plus_tagged(
            text, entities, max_length=max_length
        )
        encoding['intent_label'] = sample['intent']
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)
    return dataset_for_loader

# トークナイザのロード
# 固有表現のカテゴリーの数`num_entity_type`を入力に入れる必要がある。
tokenizer = NER_tokenizer_BIO.from_pretrained(
    MODEL_NAME,
    num_entity_type=NUM_ENTITY_TYPE
)

# データセットの作成
max_length = 128
dataset_train_for_loader = create_dataset(
    tokenizer, dataset_train, max_length
)
dataset_val_for_loader = create_dataset(
    tokenizer, dataset_val, max_length
)

# データローダの作成
dataloader_train = DataLoader(
    dataset_train_for_loader, batch_size=32, shuffle=True
)
dataloader_val  = DataLoader(dataset_val_for_loader, batch_size=256)

In [ ]:
i_data = 2
print(dataset_train[i_data])
tokens = tokenizer.tokenize(dataset_train[i_data]['text'])
print(tokens)
print(dataset_train_for_loader[i_data])

In [ ]:
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='./model',
)

# 学習の方法を指定
trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks = [checkpoint]
)

In [ ]:
# PyTorch Lightningのモデルのロード
num_slot_labels = 2*NUM_ENTITY_TYPE+1
model = BertForTokenClassification_pl(
    MODEL_NAME, num_slot_labels=num_slot_labels, num_intent_labels=NUM_INTENT_LABELS, lr=1e-5
)

# ファインチューニング
trainer.fit(model, dataloader_train, dataloader_val)
best_model_path = checkpoint.best_model_path

In [ ]:
def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    """
    正解と予測を比較し、モデルの固有表現抽出の性能を評価する。
    type_idがNoneのときは、全ての固有表現のタイプに対して評価する。
    type_idが整数を指定すると、その固有表現のタイプのIDに対して評価を行う。
    """
    num_entities    = 0 # 固有表現(正解)の個数
    num_predictions = 0 # BERTにより予測された固有表現の個数
    num_correct     = 0 # BERTにより予測のうち正解であった固有表現の数
    indices_incorredt = []
    
    # それぞれの文章で予測と正解を比較。
    # 予測は文章中の位置とタイプIDが一致すれば正解とみなす。
    counter = 0
    for entities, entities_predicted \
        in zip(entities_list, entities_predicted_list):

        if type_id:
            entities = [ e for e in entities if e['type_id'] == type_id ]
            entities_predicted = [ 
                e for e in entities_predicted if e['type_id'] == type_id
            ]
            
        get_span_type = lambda e: (e['span'][0], e['type_id'])
        set_entities = set( get_span_type(e) for e in entities )
        set_entities_predicted = \
            set( get_span_type(e) for e in entities_predicted )

        num_entities += len(entities)
        num_predictions += len(entities_predicted)
        num_correct += len( set_entities & set_entities_predicted )
        
        # debug
        if(len(set_entities) != len( set_entities & set_entities_predicted )):
            indices_incorredt.append(counter)
        #    print(set_entities)
        #    print(set_entities_predicted)

        counter += 1
    
    # 指標を計算
    precision = num_correct/num_predictions # 適合率
    recall = num_correct/num_entities # 再現率
    f_value = 2*precision*recall/(precision+recall) # F値

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    print(indices_incorredt)
    return result

In [ ]:
# 性能評価
model = BertForTokenClassification_pl.load_from_checkpoint(
    best_model_path
)
model.eval()
bert_tc.eval()
bert_tc = model.bert_tc.cuda()

In [ ]:
intents_list            = [] # 正解インテントを追加している
intents_predicted_list  = [] # 分類されたインテントを追加していく
entities_list           = [] # 正解の固有表現を追加していく
entities_predicted_list = [] # 抽出された固有表現を追加していく

for sample in tqdm(dataset_test):
    text = sample['text']
    encoding, spans = tokenizer.encode_plus_untagged(
        text, max_length=128, return_tensors='pt'
    )
    encoding = { k: v.cuda() for k, v in encoding.items() } 

    with torch.no_grad():
        total_loss, logits_intent, logits_slot = bert_tc(**encoding)
        scores_intent = logits_intent.cpu().numpy()
        scores_slots  = logits_slot[0].cpu().numpy().tolist()

    # 分類スコアを固有表現に変換する
    entities_predicted = tokenizer.convert_bert_output_to_entities(
        text, scores_slots, spans
    )
    
    intents_list.append(sample['intent'])
    intents_predicted_list.append(scores_intent.argmax(-1)[0])
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

In [ ]:
# インテント分類スコア
counter = 0.0
for pred, truth in zip(intents_predicted_list, intents_list):
    counter += float(pred == truth)
print('intent classification accuracy = ', counter/len(intents_list))
# 固有表現抽出スコア
print(evaluate_model(entities_list, entities_predicted_list))

In [ ]:
# 個別に実行
entities           = [] # 正解の固有表現
entities_predicted = [] # 抽出された固有表現

# ツンデレを検索
#sample = next((x for x in dataset_test if ('いいんだからね' in x['text'])), None)
sample = dataset_test[75]

text = sample['text']
encoding, spans = tokenizer.encode_plus_untagged(
    text, max_length=128, return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items() } 

with torch.no_grad():
    total_loss, logits_intent, logits_slot = bert_tc(**encoding)
    scores_intent = logits_intent.cpu().numpy()
    scores_slots  = logits_slot[0].cpu().numpy().tolist()

# Intent 分類スコアを Intent に変換する
intent = scores_intent.argmax(-1)[0]
# Slot 分類スコアを固有表現に変換する
entities_predicted = tokenizer.convert_bert_output_to_entities(
    text, scores_slots, spans
)

print("正解", sample)
print("予測 intent  :", intent)
print("予測 entities:", entities_predicted)

In [ ]:
# 注意事項。"白い" は 1 token
tokens = tokenizer.tokenize('ラボの白いライトオンに')
print(tokens)

In [ ]:
tokenizer.save_pretrained("./model/iot-nlu-tokenizer")